<a href="https://colab.research.google.com/github/cemreefe/OyVeOtesi-parser/blob/main/parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the file path and content
%cd '/content/drive/MyDrive/'
%mkdir 'ballot-results-oyveotesi'
%cd 'ballot-results-oyveotesi'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
mkdir: cannot create directory ‘ballot-results-oyveotesi’: File exists
/content/drive/MyDrive/ballot-results-oyveotesi


In [ ]:
parse_complete = {
    "title": False,
    "cities": [],
    "districts": [],
    "neighborhoods": [],
    "schools": [],
    "ballots": []
}

In [ ]:
import requests
import os
import csv
import time

# API base URL
base_url = "https://api-sonuc.oyveotesi.org/api/v1"

# Create a directory for storing images
image_directory = "school_images"
os.makedirs(image_directory, exist_ok=True)

CSV_SEP = ","

# Function to download an image
def download_image(image_url, image_path):
    response = requests.get(image_url)
    
    # Create directories if they don't exist
    os.makedirs(os.path.dirname(image_path), exist_ok=True)
    
    with open(image_path, "wb") as file:
        file.write(response.content)

# Function to write data to the CSV file
def write_to_csv(data):
    data = f"{CSV_SEP}".join(map(lambda x: str(x), data)) + "\n"
    csv_file_path = "school_data.csv"
    with open(csv_file_path, "a") as csvfile:
        csvfile.write(data)

def map_votes(data, num):
  votes_csv = ""
  for i in range(num):
    key = f"{i+1}"
    datum = 0 if key not in data else data[key]
    votes_csv += f"{datum}{CSV_SEP}"
  votes_csv = votes_csv[:-1]
  return votes_csv

if not parse_complete['title']:
  # Create a CSV file to store the school data
  write_to_csv(["City", "District", "School ID", "School Name", "Ballot Box", "Submission ID", "Image URL (Presidential)", "Image Path (Presidential)",
                "Total Votes (Presidential)", "Erdogan Votes (Presidential)", "Ince Votes (Presidential)", "Kilicdaroglu Votes (Presidential)", "Ogan Votes (Presidential)",
                "Image URL (General)", "Image Path (General)", "Total Votes (General)",
                f"Millet Partisi{CSV_SEP}Hak Ve Özgürlükler Partisi{CSV_SEP}Türkiye Komünist Partisi{CSV_SEP}Türkiye Komünist Hareketi{CSV_SEP}Sol Parti{CSV_SEP}Genç Parti{CSV_SEP}Memleket Partisi{CSV_SEP}Büyük Birlik Partisi{CSV_SEP}Adalet Ve Kalkınma Partisi{CSV_SEP}Yeniden Refah Partisi{CSV_SEP}Milliyetçi Hareket Partisi{CSV_SEP}Yeşiller Ve Sol Gelecek Partisi{CSV_SEP}Türkiye İşçi Partisi{CSV_SEP}Adalet Birlik Partisi{CSV_SEP}Anavatan Partisi{CSV_SEP}Yenilik Partisi{CSV_SEP}Halkın Kurtuluş Partisi{CSV_SEP}Milli Yol Partisi{CSV_SEP}Vatan Partisi{CSV_SEP}Güç Birliği Partisi{CSV_SEP}Cumhuriyet Halk Partisi{CSV_SEP}İyi Parti{CSV_SEP}Adalet Partisi{CSV_SEP}Zafer Partisi"])

parse_complete['title'] = True


# Retrieve data for all cities
for city_id in range(1, 83):
    # Get the districts for the current city
    districts_url = f"{base_url}/cities/{city_id}/districts"
    district_response = requests.get(districts_url)
    districts = district_response.json()

    print("- City-" + str(city_id))

    if city_id in parse_complete['cities']:
      print("<City already parsed>")
      continue


    # Retrieve data for all districts
    for district in districts:
        district_id = district["id"]
        district_name = district["name"]

        print("  - " + district_name)

        if district_id in parse_complete['districts']:
          print("<District already parsed>")
          continue

        # Get the neighborhoods for the current district
        neighborhoods_url = f"{base_url}/cities/{city_id}/districts/{district_id}/neighborhoods"
        neighborhood_response = requests.get(neighborhoods_url)
        neighborhoods = neighborhood_response.json()

        # Retrieve data for all neighborhoods
        for neighborhood in neighborhoods:
            neighborhood_id = neighborhood["id"]
            neighborhood_name = neighborhood["name"]

            print("    - " + neighborhood_name)

            if neighborhood_id in parse_complete['neighborhoods']:
              print("<Neighborhood already parsed>")
              continue

            # Get the schools for the current neighborhood
            schools_url = f"{base_url}/cities/{city_id}/districts/{district_id}/neighborhoods/{neighborhood_id}/schools"
            schools_response = requests.get(schools_url)
            schools = schools_response.json()

            # Retrieve data for all schools
            for school in schools:
                school_id = school["id"]
                school_name = school["name"]

                print("      - " + str(school_id) + ": " + school_name)

                if school_id in parse_complete['schools']:
                  print("<School already parsed>")
                  continue

                # Get the school details
                school_details_url = f"{base_url}/submission/school/{school_id}"
                school_details_response = requests.get(school_details_url)
                school_details = school_details_response.json()

                # Wait for a few seconds
                time.sleep(1)

                # Iterate over each school detail entry
                for detail in school_details:
                    ballot_box_number = detail["ballot_box_number"]
                    submission_id = detail["submission_id"] if "submission_id" in detail else "?"

                    if (f"{school_id}/{ballot_box_number}") in parse_complete['ballots']:
                      print("<Ballot already parsed>")
                      continue

                    cm_result = detail["cm_result"]
                    mv_result = detail["mv_result"]

                    if not cm_result:
                      print("No data for cm")
                      cm_image_url = "?"
                      cm_image_path = "?"
                      cm_total_vote = "?"
                      cm_votes = {}
                    else:
                      # Download the CM image
                      cm_image_url = cm_result["image_url"]
                      cm_image_path = f"{image_directory}/{city_id}/{district_id}/{school_id}_cm.jpg"
                      download_image(cm_image_url, cm_image_path)
                      cm_total_vote = cm_result["total_vote"]
                      cm_votes = cm_result["votes"]

                    if not mv_result:
                      print("No data for mv")
                      mv_image_url = "?"
                      mv_image_path = "?"
                      mv_total_vote = "?"
                      mv_votes = {}
                    else:
                      # Download the MV image
                      mv_image_url = mv_result["image_url"]
                      mv_image_path = f"{image_directory}/{city_id}/{district_id}/{school_id}_mv.jpg"
                      download_image(mv_image_url, mv_image_path)
                      mv_total_vote = mv_result["total_vote"]
                      mv_votes = mv_result["votes"]

                    # Write the data to the CSV file
                    write_to_csv([
                        city_id, district_name, school_id, school_name, ballot_box_number, submission_id,
                        cm_image_url, cm_image_path,
                        cm_total_vote, map_votes(cm_votes, 4),
                        mv_image_url, mv_image_path,
                        mv_total_vote, map_votes(mv_votes, 24)
                    ])

                    # Wait for a few seconds
                    #time.sleep(0.5)
                    
                    # after finishing ballot    
                    parse_complete['ballots'].append(f"{school_id}/{ballot_box_number}")
                # after finishing school
                parse_complete['schools'].append(school_id)
            # Wait for a few seconds after finishing neighborhood
            time.sleep(1)
            parse_complete['neighborhoods'].append(neighborhood_id)
        # Wait for a few seconds after finishing district
        time.sleep(1)
        parse_complete['districts'].append(district_id)
    # Wait for a few seconds after finishing city
    time.sleep(1)
    parse_complete['cities'].append(city_id)

print("Data retrieval and image download complete.")
print("CSV file generated.")

- City-1
  - AYANCIK
    - ABDÜLKADİRKÖY
      - 389: KÖY KONAĞI
No data for cm
No data for mv
    - AĞAÇLI KÖYÜ
      - 406: AĞAÇLI KÖY BİNASI
No data for cm
No data for mv
    - AKÇAKESE KÖYÜ
      - 355: AKÇAKESE KÖYÜ İLKOKULU
No data for cm
No data for mv
    - AKÖREN KÖYÜ
      - 398: AKÖREN KÖYÜ İLKOKULU
No data for cm
No data for mv
    - ALİKÖY
      - 365: ALİ KÖYÜ İLKOKULU
No data for cm
No data for mv
    - ARMUTLUYAZI KÖYÜ
      - 414: ARMUTLUYAZI KÖYÜ İLKOKULU
No data for cm
No data for mv
    - AŞAĞIKÖY
      - 371: AŞAĞI KÖYÜ İLKOKULU
No data for cm
No data for mv
    - AVDULLU KÖYÜ
      - 366: AVDULLU KÖYÜ İLKOKULU
No data for cm
No data for mv
    - AYGÖRDÜ KÖYÜ
      - 381: KÖY CAMİİ
No data for cm
No data for mv
    - BABAÇAY KÖYÜ
      - 391: BABAÇAY KÖYÜ ORM. İŞ. M. L.
No data for cm
No data for mv
    - BABAKÖY
      - 399: BABA KÖYÜ İLKOKULU
No data for cm
No data for mv
    - BAHÇELİ KÖYÜ
      - 407: BAHÇELİ KÖYÜ İLKOKULU
No data for cm
    - BAKIRLI KÖYÜ
    

KeyboardInterrupt: ignored